# import packages

In [1]:
import numpy as np
import os, glob
from netCDF4 import Dataset
import xarray, wrf
from pandas import to_datetime
from scipy.signal import welch
from scipy import fftpack
from matplotlib.cm import get_cmap
import cartopy.crs as crs
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter

In [2]:
fs=14
plt.rc("font",size=fs)

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# define parameters

In [4]:
height = 100.0

In [5]:
dx = {"MYNN":333.0,
       "SH":333.0,
       "VLES":333.0,
       "YSU":333.0,
       "LES_25":25.0}

In [6]:
colors = {25:'k',333:'r'}
colors = {"LES_25":"k","MYNN":"#FFC300","SH":"#FF5733","YSU":"#C70039","VLES":"#900C3F"}

In [7]:
figPath = "/glade/u/home/doubrawa/figs/"

### define a time

In [8]:
year     = 2015
month    = 3

In [9]:
for day in [20]:
    directories = {"MYNN":"/glade/scratch/doubrawa/final_data/mynn/03{0}15/".format(day),
                   "SH":"/glade/scratch/doubrawa/final_data/sh/03{0}15/".format(day),
                   "VLES":"/glade/scratch/doubrawa/final_data/vles/03{0}15/".format(day),
                   "YSU":"/glade/scratch/doubrawa/final_data/ysu/03{0}15/".format(day),
                   "LES_25":"/glade/scratch/doubrawa/post_processing/"}    
    for hour in [18]:#np.arange(14,24,1):
        minutes = np.arange(0,59,10) if hour!=14 else np.arange(20,59,10)
        for minute in [50]:#minutes:
            datetime = to_datetime("{0}-{1}-{2} {3}:{4}".format(year,month,day,hour,minute),format="%Y-%m-%d %H:%M")
            print("-----------------------")
            print(datetime)
            
            w     = {}
            ws    = {}
            u     = {}
            v     = {}
            theta = {}
            xlat  = {}
            xlong = {}
            
            for key,directory in directories.items():
                
                if "LES_" not in key:
                    wrfouts = sorted(glob.glob(os.path.join(directory,'wrfout_d03_2015-03-{0}_{1}:{2:02d}*'.format(day,hour,minute))))
                    wrfout  = wrfouts[0]
                    print (os.path.split(wrfout)[-1])

                    wrfnc = Dataset(wrfout)
                    hgt   = wrf.getvar(wrfnc, "ter", units="m")
                    z     = wrf.getvar(wrfnc, "z", units="m")
                    zref  = z-hgt

                    w_3d   = wrf.getvar(wrfnc, "wa", units="m s-1", timeidx=0)
                    w[key] = wrf.interplevel(w_3d, zref, height, meta=True)                       
                    
                    theta_3d   = wrf.getvar(wrfnc, "theta", units="K", timeidx=0)
                    theta[key] = wrf.interplevel(theta_3d, zref, height, meta=True)                       

                    xlat[key]  = wrf.getvar(wrfnc, "lat")
                    xlong[key] = wrf.getvar(wrfnc, "lon")
                    
                    ua = wrf.getvar(wrfnc,"ua", timeidx=0)
                    va = wrf.getvar(wrfnc,"va", timeidx=0)
                    u[key] = wrf.interplevel(ua, zref, height, meta=True)
                    v[key] = wrf.interplevel(va, zref, height, meta=True)
                    ws[key] = (u[key]**2+v[key]**2)**0.5                    

                else:                    
                    wrfouts = sorted(glob.glob(os.path.join(directory,'WRF_LES_25m_{0}*2015-03-{1}_{2}:{3:02d}*'.format(height,day,hour,minute))))
                    wrfout  = wrfouts[0]
                    print (os.path.split(wrfout)[-1])                
                    
                    wrfnc = xarray.open_dataset(wrfout)
                    w[key] = wrfnc["w"]
                    theta[key] = wrfnc["theta"]
                    u[key] = wrfnc['u']
                    v[key] = wrfnc['v']
                    ws[key] = (wrfnc['u']**2+wrfnc['v']**2)**0.5
                    xlat[key]  = wrfnc["lat"]
                    xlong[key] = wrfnc["lon"]


            # equivalent x/y coordinates across domains

            x = {}
            y = {}

            ref_wrfnc = Dataset("/glade/scratch/doubrawa/final_data/les/032815/032815_14UTC/wrfout_d04_2015-03-28_14:00:00_0000")

            for key in directories.keys():
                print (key)
                x[key], y[key] = wrf.ll_to_xy(ref_wrfnc, xlat[key], xlong[key])
                x[key]   = np.reshape(x[key].data,w[key].shape)
                y[key]   = np.reshape(y[key].data,w[key].shape)

            # clip all of them to the same area

            xmin, xmax = np.min(x["LES_25"]), np.max(x["LES_25"])
            ymin, ymax = np.min(y["LES_25"]), np.max(y["LES_25"])

            x_clipped     = {}
            y_clipped     = {}
            w_clipped     = {}
            u_clipped     = {}
            v_clipped     = {}
            theta_clipped = {}
            ws_clipped    = {}

            for key in ["MYNN","SH","YSU","VLES"]:

                print (key)

                condition_x = ( (x[key] >= xmin) & (x[key]<=xmax) )
                condition_y = ( (y[key] >= ymin) & (y[key]<=ymax) )
                condition   = condition_x & condition_y

                idx_sn, idx_we = np.where(condition)
                idx_sn = np.unique(idx_sn)
                idx_we = np.unique(idx_we)

                w_clipped[key] = (w[key].isel(west_east=idx_we,south_north=idx_sn)).copy()
                u_clipped[key] = (u[key].isel(west_east=idx_we,south_north=idx_sn)).copy()
                v_clipped[key] = (v[key].isel(west_east=idx_we,south_north=idx_sn)).copy()    
                ws_clipped[key] = (ws[key].isel(west_east=idx_we,south_north=idx_sn)).copy()
                theta_clipped[key] = (theta[key].isel(west_east=idx_we,south_north=idx_sn)).copy()
                x_clipped[key] = (x[key][idx_sn,idx_we]).copy()
                y_clipped[key] = (y[key][idx_sn,idx_we]).copy()

            x_clipped["LES_25"] = x["LES_25"]
            y_clipped["LES_25"] = y["LES_25"]
            w_clipped["LES_25"] = w["LES_25"]
            u_clipped["LES_25"] = u["LES_25"]
            v_clipped["LES_25"] = v["LES_25"]
            theta_clipped["LES_25"] = theta["LES_25"]
            ws_clipped["LES_25"] = ws["LES_25"]

#             # vertical velocity

#             levels = np.arange(-1.0,1.1,0.1)
#             fig   = plt.figure(figsize=(18,5))

#             axbig = fig.add_subplot(111)
#             axbig.spines['top'].set_color('none')
#             axbig.spines['bottom'].set_color('none')
#             axbig.spines['left'].set_color('none')
#             axbig.spines['right'].set_color('none')
#             axbig.tick_params(labelcolor='none', top='off', bottom='off', left='off', right='off')
#             axbig.set_xlabel("East-West Distance [km]",labelpad=-40)
#             axbig.set_ylabel("South-North Distance [km]",labelpad=5)

#             ax  = {}
#             for iax,key in enumerate(["LES_25","VLES","MYNN","YSU","SH"]):

#                 if iax==0:
#                     ax[iax] = fig.add_subplot(1,5,iax+1,aspect='equal')
#                 else:
#                     ax[iax] = fig.add_subplot(1,5,iax+1,sharex=ax[0],sharey=ax[0],aspect='equal')

#                 key_title = "LES" if key=="LES_25" else key

#                 ax[iax].set_title(key_title,fontsize=fs)

#                 contour = np.squeeze(w_clipped[key])
#                 xx      = np.squeeze(x_clipped[key])
#                 yy      = np.squeeze(y_clipped[key])

#                 p = ax[iax].contourf(xx,yy,contour,levels=levels,cmap="RdBu")
#                 #p = ax[iax].pcolormesh(xx,yy,contour,vmin=np.min(levels),vmax=np.max(levels),cmap="RdBu")        

#                 ax[iax].set_xlim([xmin,xmax])
#                 ax[iax].set_ylim([ymin,ymax])    

#                 if iax>0:
#                     plt.setp(ax[iax].get_yticklabels(), visible=False)

#                 ax[iax].xaxis.set_tick_params(direction='in')
#                 ax[iax].yaxis.set_tick_params(direction='in')           

#                 fig.subplots_adjust(wspace=0.1)

#                 desired_ticks = [0,15,30]
#                 xticks        = [(ii*1000)/25.0 for ii in desired_ticks]
#                 ax[iax].set_xticks(xticks)
#                 labels = ["{0:.0f}".format(dx) for dx in desired_ticks]
#                 ax[iax].set_xticklabels(labels,fontsize=fs)

#             ax[0].set_yticks(xticks)
#             ax[0].set_yticklabels(labels,fontsize=fs)   

#             fig.subplots_adjust(right=0.85)
#             cbar_ax = fig.add_axes([0.86, 0.25, 0.01, 0.51])
#             clb = fig.colorbar(p, cax=cbar_ax, format=FormatStrFormatter('%4.1f'))
#             clb.set_label('Vertical Velocity $w$ [m/s]', labelpad=10)

#             fig.savefig(os.path.join(figPath,"contours_w_clipped_{0:%Y-%m-%d-%H-%M}_{1:.0f}m.png".format(datetime,height)),bbox_to_inches="tight",pad_inches = 0)    

            # temperature

            levels = np.arange(302,312,1)
            fig   = plt.figure(figsize=(18,5))

            axbig = fig.add_subplot(111)
            axbig.spines['top'].set_color('none')
            axbig.spines['bottom'].set_color('none')
            axbig.spines['left'].set_color('none')
            axbig.spines['right'].set_color('none')
            axbig.tick_params(labelcolor='none', top='off', bottom='off', left='off', right='off')
            axbig.set_xlabel("East-West Distance [km]",labelpad=-40)
            axbig.set_ylabel("South-North Distance [km]",labelpad=5)

            ax  = {}
            for iax,key in enumerate(["LES_25","VLES","MYNN","YSU","SH"]):

                if iax==0:
                    ax[iax] = fig.add_subplot(1,5,iax+1,aspect='equal')
                else:
                    ax[iax] = fig.add_subplot(1,5,iax+1,sharex=ax[0],sharey=ax[0],aspect='equal')

                key_title = "LES" if key=="LES_25" else key

                ax[iax].set_title(key_title,fontsize=fs)

                contour = np.squeeze(theta_clipped[key])
                xx      = np.squeeze(x_clipped[key])
                yy      = np.squeeze(y_clipped[key])

                p = ax[iax].pcolormesh(xx,yy,contour,vmin=np.min(levels),vmax=np.max(levels),cmap="magma")    

                ax[iax].set_xlim([xmin,xmax])
                ax[iax].set_ylim([ymin,ymax])    

                if iax>0:
                    plt.setp(ax[iax].get_yticklabels(), visible=False)

                ax[iax].xaxis.set_tick_params(direction='in')
                ax[iax].yaxis.set_tick_params(direction='in')           

                fig.subplots_adjust(wspace=0.1)

                desired_ticks = [0,15,30]
                xticks        = [(ii*1000)/25.0 for ii in desired_ticks]
                ax[iax].set_xticks(xticks)
                labels = ["{0:.0f}".format(dx) for dx in desired_ticks]
                ax[iax].set_xticklabels(labels,fontsize=fs)

            ax[0].set_yticks(xticks)
            ax[0].set_yticklabels(labels,fontsize=fs)   

            fig.subplots_adjust(right=0.85)
            cbar_ax = fig.add_axes([0.86, 0.25, 0.01, 0.51])
            clb = fig.colorbar(p, cax=cbar_ax, format=FormatStrFormatter('%3.0f'))
            clb.set_label(r'Potential Temperature $\theta$ [K]', labelpad=10)

#             fig.savefig(os.path.join(figPath,"contours_theta_clipped_{0:%Y-%m-%d-%H-%M}_{1:.0f}m.png".format(datetime,height)),bbox_to_inches="tight",pad_inches = 0)    


            # wind speed
            levels = np.arange(0,15,1)
            fig   = plt.figure(figsize=(18,5))

            axbig = fig.add_subplot(111)
            axbig.spines['top'].set_color('none')
            axbig.spines['bottom'].set_color('none')
            axbig.spines['left'].set_color('none')
            axbig.spines['right'].set_color('none')
            axbig.tick_params(labelcolor='none', top='off', bottom='off', left='off', right='off')
            axbig.set_xlabel("East-West Distance [km]",labelpad=-40)
            axbig.set_ylabel("South-North Distance [km]",labelpad=5)

            ax  = {}
            for iax,key in enumerate(["LES_25","VLES","MYNN","YSU","SH"]):

                if iax==0:
                    ax[iax] = fig.add_subplot(1,5,iax+1,aspect='equal')
                else:
                    ax[iax] = fig.add_subplot(1,5,iax+1,sharex=ax[0],sharey=ax[0],aspect='equal')

                key_title = "LES" if key=="LES_25" else key

                ax[iax].set_title(key_title,fontsize=fs)

                contour = np.squeeze(ws_clipped[key])
                xx      = np.squeeze(x_clipped[key])
                yy      = np.squeeze(y_clipped[key]) 

                if xx.ndim==1:
                    [xx,yy]=np.meshgrid(xx,yy)

            #     p = ax[iax].contourf(xx,yy,contour,levels=levels,cmap="gnuplot")
                p = ax[iax].pcolormesh(xx,yy,contour,vmin=np.min(levels),vmax=np.max(levels),cmap="winter")    


                ax[iax].set_xlim([xmin,xmax])
                ax[iax].set_ylim([ymin,ymax])    

                if iax>0:
                    plt.setp(ax[iax].get_yticklabels(), visible=False)

                ax[iax].xaxis.set_tick_params(direction='in')
                ax[iax].yaxis.set_tick_params(direction='in')           

                fig.subplots_adjust(wspace=0.1)

                desired_ticks = [0,15,30]
                xticks        = [(ii*1000)/25.0 for ii in desired_ticks]
                ax[iax].set_xticks(xticks)
                labels = ["{0:.0f}".format(dx) for dx in desired_ticks]
                ax[iax].set_xticklabels(labels,fontsize=fs)   

            ax[0].set_yticks(xticks)
            ax[0].set_yticklabels(labels,fontsize=fs)   

            fig.subplots_adjust(right=0.85)
            cbar_ax = fig.add_axes([0.86, 0.25, 0.01, 0.51])
            clb = fig.colorbar(p, cax=cbar_ax, format=FormatStrFormatter('%4.1f'))
            clb.set_label(r'Wind Speed $U$ [m/s]', labelpad=10)

#             fig.savefig(os.path.join(figPath,"contours_ws_clipped_{0:%Y-%m-%d-%H-%M}_{1:.0f}m.png".format(datetime,height)),bbox_to_inches="tight",pad_inches = 0)    
            
            # wind speed with quiver
            levels = np.arange(0,15,1)
            fig   = plt.figure(figsize=(18,5))

            axbig = fig.add_subplot(111)
            axbig.spines['top'].set_color('none')
            axbig.spines['bottom'].set_color('none')
            axbig.spines['left'].set_color('none')
            axbig.spines['right'].set_color('none')
            axbig.tick_params(labelcolor='none', top='off', bottom='off', left='off', right='off')
            axbig.set_xlabel("East-West Distance [km]",labelpad=-40)
            axbig.set_ylabel("South-North Distance [km]",labelpad=5)

            ax  = {}
            for iax,key in enumerate(["LES_25","VLES","MYNN","YSU","SH"]):

                if iax==0:
                    ax[iax] = fig.add_subplot(1,5,iax+1,aspect='equal')
                else:
                    ax[iax] = fig.add_subplot(1,5,iax+1,sharex=ax[0],sharey=ax[0],aspect='equal')

                key_title = "LES" if key=="LES_25" else key

                ax[iax].set_title(key_title,fontsize=fs)

                contour = np.squeeze(ws_clipped[key])
                xx      = np.squeeze(x_clipped[key])
                yy      = np.squeeze(y_clipped[key])
                uu      = np.squeeze(u_clipped[key])
                vv      = np.squeeze(v_clipped[key])    

                if xx.ndim==1:
                    [xx,yy]=np.meshgrid(xx,yy)

                p = ax[iax].pcolormesh(xx,yy,contour,vmin=np.min(levels),vmax=np.max(levels),cmap="winter")    

                if key=="LES_25":
                    skip = (slice(None,None,100),slice(None,None,100))
                else:
                    skip = (slice(None,None,10),slice(None,None,10))

                Q = ax[iax].quiver(xx[skip],yy[skip],uu[skip], vv[skip], color='w',units='xy',scale_units='inches',scale=50)    

                ax[iax].set_xlim([xmin,xmax])
                ax[iax].set_ylim([ymin,ymax])    

                if iax>0:
                    plt.setp(ax[iax].get_yticklabels(), visible=False)

                ax[iax].xaxis.set_tick_params(direction='in')
                ax[iax].yaxis.set_tick_params(direction='in')           

                fig.subplots_adjust(wspace=0.1)

                desired_ticks = [0,15,30]
                xticks        = [(ii*1000)/25.0 for ii in desired_ticks]
                ax[iax].set_xticks(xticks)
                labels = ["{0:.0f}".format(dx) for dx in desired_ticks]
                ax[iax].set_xticklabels(labels,fontsize=fs)   

            ax[0].set_yticks(xticks)
            ax[0].set_yticklabels(labels,fontsize=fs)   

            fig.subplots_adjust(right=0.85)
            cbar_ax = fig.add_axes([0.86, 0.25, 0.01, 0.51])
            clb = fig.colorbar(p, cax=cbar_ax, format=FormatStrFormatter('%4.1f'))
            clb.set_label(r'Wind Speed $U$ [m/s]', labelpad=10)

            qk = ax[iax].quiverkey(Q, 0.87, 0.80, 15, "15 m/s", color='k', labelcolor='k',labelpos='E',coordinates='figure')    

#             fig.savefig(os.path.join(figPath,"contours_ws_and_arrows_clipped_{0:%Y-%m-%d-%H-%M}_{1:.0f}m.png".format(datetime,height)),bbox_to_inches="tight",pad_inches = 0)                

-----------------------
2015-03-20 18:50:00
wrfout_d03_2015-03-20_18:50:00
wrfout_d03_2015-03-20_18:50:00
wrfout_d03_2015-03-20_18:50:00
wrfout_d03_2015-03-20_18:50:00


IndexError: list index out of range